In [1]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.chart import BarChart, Reference
import string



In [11]:
def excel_auto(file_name):
    
    # ler o arquivo excel
    excel_f = pd.read_excel(file_name)
    
    # pivot table
    tabela1 = excel_f.pivot_table(index='Gender', columns='Product line', values='Total', aggfunc='sum').round(0)
    
    # separando o nome do arquivo da sua extensao
    mes_extensao = file_name.split('_')[1]
    
    # salvando o arquivo como excel
    tabela1.to_excel(f'relatorio_{mes_extensao}', sheet_name='vendas',startrow=4)
    
    # load no workbook e selecionando a aba
    wb = load_workbook(f'relatorio_{mes_extensao}')
    sheet = wb['vendas']
    
    # referencia das celulas
    min_column = wb.active.min_column
    max_column = wb.active.max_column
    min_row = wb.active.min_row
    max_row = wb.active.max_row
    
    # adicionando o grafico de barras
    barchart = BarChart()
    data = Reference(sheet,
                     min_col=min_column+1,
                     max_col=max_column,
                     min_row=min_row,
                     max_row=max_row) 
    categories = Reference(sheet,
                           min_col=min_column,
                           max_col=min_column,
                           min_row=min_row+1,
                           max_row=max_row) 
    
    # adicionando os dados e categorias
    barchart.add_data(data, titles_from_data=True)
    barchart.set_categories(categories)
    
    sheet.add_chart(barchart, "B12") #location chart
    barchart.title = 'Vendas por tipo de produto'
    barchart.style = 10  #choose the chart style
    
    # aplicando a formula (soma)
    # criando o alfabeto para usar como referencia
    
    alphabet = list(string.ascii_uppercase)
    excel_alphabet = alphabet[0:max_column] 
    # somatoria nas colunas B-G
    for i in excel_alphabet:
        if i!='A':
            sheet[f'{i}{max_row+1}'] = f'=SUM({i}{min_row+1}:{i}{max_row})' 
            sheet[f'{i}{max_row+1}'].number_format = 'R$ #,##0.00'
    sheet[f'{excel_alphabet[0]}{max_row+1}'] = 'Total'
    
    # pegando o nome do mes/ano
    mes = mes_extensao.split('.')[0]
    
    # formatação do relatorio
    sheet['A1'] = 'Relatório de Vendas'
    sheet['A2'] = '2021'
    sheet['A1'].font = Font('Arial', bold=True, size=20)
    sheet['A2'].font = Font('Arial', bold=True, size=10)
    wb.save(f'relatorio_{mes_extensao}')
    return 

In [12]:
excel_auto('sales_2021.xlsx')

In [13]:
excel_auto('sales_january.xlsx')
excel_auto('sales_february.xlsx')
excel_auto('sales_march.xlsx')